In [10]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.datasets import mnist
import matplotlib.pyplot as plt

kernel_size = 8

def generator_model():
    dropout = 0.25
    dim = 7
    depth = 128
    # In: 100
    # Out: 7 x 7 x 128
    model = Sequential()
    model.add(Dense(dim*dim*depth, input_dim=100))
    model.add(LeakyReLU())
    model.add(Reshape((dim, dim, depth)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dropout(dropout))
    # In: 7 x 7 x 128
    # Out: 14 x 14 x 64
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/2), kernel_size, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU())
    # In: 14 x 14 x 64
    # Out: 28 x 28 x 32
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/4), kernel_size, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU())
    # In: 28 x 28 x 32
    # Out: 28 x 28 x 1
#     model.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
#     model.add(BatchNormalization(momentum=0.8))
#     model.add(LeakyReLU())
    model.add(Conv2DTranspose(1, kernel_size, padding='same'))
    model.add(Activation('tanh'))
    model.summary()
    return model


def discriminator_model():
    depth = 64
    dropout = 0.25
    model = Sequential()
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth = 64
    model.add(Conv2D(depth, kernel_size, strides = 2, padding ='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*2, kernel_size, strides = 2, padding ='same'))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*4, kernel_size, strides = 2, padding ='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*8, kernel_size, strides = 1, padding ='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    return model

def adversial_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

In [11]:
def train(epochs = 1000, batch_size = 128, save_interval = 50):
    latent_dim = 100
    img_cols, img_rows = 28, 28
    # Initialize data and models
    (X_train, y_train), (X_test, y_test) = mnist.load_data() 
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)
    discriminator = discriminator_model()
    generator = generator_model()
    adversial = adversial_model(generator, discriminator) # discriminator is frozen
    g_optim = Adam(lr=0.0002, beta_1=0.5)
    d_optim = Adam(lr=0.0002, beta_1=0.5)
#     generator.compile(loss='binary_crossentropy', optimizer="RMSprop")
    adversial.compile(loss='binary_crossentropy', optimizer=g_optim, metrics=['accuracy'])
#     discriminator.trainable = True # unfreeze discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim, metrics=['accuracy'])
    
    # labels
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # Stuff for saving images
    filename = 'mnist.png'
    noise_input = None
    if save_interval > 0:
        noise_input = np.random.uniform(-1, 1, size=[16, latent_dim])
    
    # The training loop
    for epoch in range(epochs):
        # Select a random half of images
        index = np.random.randint(0, X_train.shape[0], batch_size)
        real_images = X_train[index]

        # Sample noise and generate a batch of new images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_images = generator.predict(noise)
        
        # Train on soft labels (add noise to labels as well)
        noise_prop = 0.05 # Randomly flip 5% of labels

        # Prepare labels for real data
#         real = np.zeros((batch_size, 1)) + np.random.uniform(low=0.0, high=0.1, size=(batch_size, 1))
#         flipped_idx = np.random.choice(np.arange(len(real)), size=int(noise_prop*len(real)))
#         real[flipped_idx] = 1 - real[flipped_idx]

        d_loss_real = discriminator.train_on_batch(real_images, real)
        
        # Prepare labels for fake data
#         fake = np.ones((batch_size, 1)) - np.random.uniform(low=0.0, high=0.1, size=(batch_size, 1))
#         flipped_idx = np.random.choice(np.arange(len(fake)), size=int(noise_prop*len(fake)))
#         fake[flipped_idx] = 1 - fake[flipped_idx]
        
        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
#         real_images = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]
#         noise = np.random.uniform(-1, 1, size=[batch_size, latent_dim])
#         fake_images = generator.predict(noise)
#         combined_images = np.concatenate((real_images, fake_images))
#         labels = np.ones([2*batch_size, 1])
#         labels[batch_size:, :] = 0
#         labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
#         labels += 0.05 * np.random.random(labels.shape)
#         d_loss = discriminator.train_on_batch(combined_images, labels)
                                        
#         misleading_targets = np.ones([batch_size, 1])
        random_latent_vectors = np.random.normal(0, 1, (batch_size, latent_dim))
        real = np.zeros((batch_size, 1))
#         discriminator.trainable = False # freeze the discriminator
        a_loss = adversial.train_on_batch(random_latent_vectors, real)
#         discriminator.trainable = True # unfreeze the discrminator
       
    
        # Plot the progress
#         print ("%d [D loss: %f, acc.: %.2f%%] [A loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], a_loss))
        log_mesg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
                       
        # Saving sample generated images
        if save_interval>0:
            step = epoch+1
            if step%save_interval==0:
                filename = "mnist_%d.png" % step
                images = generator.predict(noise_input)
                plt.figure(figsize=(10,10))
                for i in range(images.shape[0]):
                    plt.subplot(4, 4, i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image, [img_cols, img_rows])
                    plt.imshow(image, cmap='gray')
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig(filename)
                plt.close('all')
train(epochs = 4000, batch_size = 32, save_interval = 50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_37 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 7, 7, 128)         204928    
_________________________________________________________________
zero_padding2d_6 (ZeroPaddin (None, 8, 8, 128)         0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 8, 8, 128)         512       
_________________________________________________________________
leaky_re_lu_38 (LeakyReLU)   (None, 8, 8, 128)         0         
__________

42: [D loss: 1.069014, acc: 0.453125]  [A loss: 0.599927, acc: 0.687500]
43: [D loss: 0.935220, acc: 0.437500]  [A loss: 0.521543, acc: 0.718750]
44: [D loss: 0.958224, acc: 0.484375]  [A loss: 0.752941, acc: 0.593750]
45: [D loss: 0.893915, acc: 0.484375]  [A loss: 0.731355, acc: 0.687500]
46: [D loss: 0.836973, acc: 0.546875]  [A loss: 0.514563, acc: 0.812500]
47: [D loss: 1.037406, acc: 0.437500]  [A loss: 0.621185, acc: 0.656250]
48: [D loss: 0.932500, acc: 0.453125]  [A loss: 0.495234, acc: 0.812500]
49: [D loss: 0.959106, acc: 0.421875]  [A loss: 0.662913, acc: 0.625000]
50: [D loss: 0.931294, acc: 0.437500]  [A loss: 0.520612, acc: 0.750000]
51: [D loss: 1.030252, acc: 0.406250]  [A loss: 0.419340, acc: 0.812500]
52: [D loss: 0.946413, acc: 0.515625]  [A loss: 0.585875, acc: 0.656250]
53: [D loss: 0.911421, acc: 0.453125]  [A loss: 0.546496, acc: 0.718750]
54: [D loss: 0.957036, acc: 0.468750]  [A loss: 0.500673, acc: 0.812500]
55: [D loss: 0.827017, acc: 0.562500]  [A loss: 0.5

154: [D loss: 0.775846, acc: 0.578125]  [A loss: 0.226392, acc: 1.000000]
155: [D loss: 0.769605, acc: 0.593750]  [A loss: 0.172759, acc: 1.000000]
156: [D loss: 0.841770, acc: 0.562500]  [A loss: 0.166823, acc: 0.968750]
157: [D loss: 0.689434, acc: 0.609375]  [A loss: 0.210119, acc: 0.968750]
158: [D loss: 0.680800, acc: 0.578125]  [A loss: 0.205139, acc: 0.968750]
159: [D loss: 0.790789, acc: 0.578125]  [A loss: 0.305084, acc: 0.968750]
160: [D loss: 0.701319, acc: 0.640625]  [A loss: 0.202645, acc: 0.968750]
161: [D loss: 0.782106, acc: 0.515625]  [A loss: 0.228501, acc: 0.906250]
162: [D loss: 0.736633, acc: 0.609375]  [A loss: 0.246057, acc: 0.937500]
163: [D loss: 0.754763, acc: 0.546875]  [A loss: 0.162347, acc: 1.000000]
164: [D loss: 0.776791, acc: 0.546875]  [A loss: 0.224622, acc: 1.000000]
165: [D loss: 0.724798, acc: 0.578125]  [A loss: 0.188106, acc: 1.000000]
166: [D loss: 0.836904, acc: 0.546875]  [A loss: 0.280093, acc: 0.906250]
167: [D loss: 0.637545, acc: 0.593750]

KeyboardInterrupt: 